<a href="https://colab.research.google.com/github/Redzoro1993/ARIMA-COURSE-DATA-/blob/master/ACENIGMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
file_name = ''
df = pd.read_excel(file_name, sheet_name=0)
df['Staff Type | ST'] = df['Staff Type | ST'].str.strip()
# Extraer la fecha del nombre del archivo
date_string = file_name.split(' ')[-1].split('.')[0]

# Convertir la cadena de fecha a un objeto de fecha real (opcional, depende de tus necesidades)
# Si solo necesitas la cadena '10-19', puedes omitir este paso.
from datetime import datetime
date_object = datetime.strptime(date_string, '%m-%d')
# Si necesitas el año actual (2023), puedes agregarlo manualmente
date_object = date_object.replace(year=2023)

# Crear una nueva columna con la fecha
# Si preferiste mantener la fecha como una cadena, usa 'date_string' en lugar de 'date_object'
df['Date_File'] = date_object

# Mover la columna 'Date' al inicio del DataFrame
column_list = df.columns.tolist()
column_list.insert(0, column_list.pop(column_list.index('Date_File')))
df = df[column_list]

# Mostrar las primeras 15 filas del DataFrame
df = df.rename(columns={'Unnamed: 1': 'LOB','Unnamed: 0': 'LOB+site','CSAT | ST/QT': 'CSAT', 'Case AHT | ST/QT': 'CAHT', 'Quality Standards Adoption (IQE) % | ST' : 'QA', 'Handled | ST/QT' : 'Volumen' })
columnas_deseadas = ["Date_File", "LOB+site", "LOB", "Staff Type | ST", "CSAT", "CAHT", "QA", "Volumen"]
# Crear un nuevo DataFrame con las columnas deseadas
df = df[columnas_deseadas]


In [ ]:
valor_a_filtrar = ["AMR Tier 1 iOS Phone Spanish",
                   "AMR Tier 1 Mac+ Phone Spanish",
                   "WW Tier 1 iOS Chat English",
                   "AMR Tier 2 Mac+ Phone Spanish",
                   "WW Tier 1 Mac+ Chat English",
                   "WW Tier 1 Mac+ Chat Spanish"]

In [ ]:

# Supongamos que tienes un DataFrame llamado df
# Filtrar el DataFrame df para eliminar las filas con valores de volumen igual a 0
df = df[df['Volumen'] != 0]
df = df.dropna(subset=["LOB+site"])

valor_para_nan = -1

# Iterar a través de los valores y crear DataFrames separados para cada uno
lob_dict = {}

for valor in valor_a_filtrar:
    lob_df = df[df['LOB'] == valor]

    if lob_df.empty:
        continue

    lob_df = lob_df.drop_duplicates(subset="Staff Type | ST")

    columnas_ranking = ["CSAT", "QA"]
    for columna in columnas_ranking:
        # Crea una nueva columna para el ranking, pero mantén los NaN sin convertirlos a int
        lob_df[f"{columna} Ranking"] = lob_df[columna].rank(method='dense', ascending=False)
        # Luego, reemplaza los NaN con valor_para_nan y convierte a int
        lob_df[f"{columna} Ranking"] = lob_df[f"{columna} Ranking"].fillna(valor_para_nan).astype(int)

    # Hacer lo mismo para "CAHT"
    lob_df["CAHT Ranking"] = lob_df['CAHT'].rank(method='dense', ascending=True)
    lob_df["CAHT Ranking"] = lob_df["CAHT Ranking"].fillna(valor_para_nan).astype(int)

    # Almacenar el DataFrame filtrado en el diccionario con el valor como clave
    lob_dict[valor] = lob_df


In [ ]:
combined_df = pd.concat(lob_dict.values(), ignore_index=True)

combined_df['Total Score'] = combined_df[["CSAT Ranking", "QA Ranking", "CAHT Ranking"]].mean(axis=1)
combined_df['SRS R'] = combined_df.groupby('LOB')['Total Score'].rank(method='dense').astype(int)

combined_df

,Date_File,LOB+site,LOB,Staff Type | ST,CSAT,CAHT,QA,Volumen,CSAT Ranking,QA Ranking,CAHT Ranking,Total Score,SRS R
0,2023-10-19,AMR Tier 1 iOS Phone Spanish Apple (Austin),AMR Tier 1 iOS Phone Spanish,Apple (Austin),0.9333,16.95,NaN,185,2,-1,1,0.666667,1
1,2023-10-19,AMR Tier 1 iOS Phone Spanish Apple (Elk G...,AMR Tier 1 iOS Phone Spanish,Apple (Elk Grove),1,23.99,NaN,65,1,-1,3,1.000000,2
2,2023-10-19,AMR Tier 1 iOS Phone Spanish Concentrix (...,AMR Tier 1 iOS Phone Spanish,Concentrix (Managua),0.798,24.84,0.9213,43426,5,2,4,3.666667,4
3,2023-10-19,AMR Tier 1 iOS Phone Spanish Concentrix (...,AMR Tier 1 iOS Phone Spanish,Concentrix (San Jose HS),0.7702,27.03,0.9008,11826,6,4,6,5.333333,6
4,2023-10-19,AMR Tier 1 iOS Phone Spanish TP (Bogota),AMR Tier 1 iOS Phone Spanish,TP (Bogota),0.8228,23.91,0.922,15973,3,1,2,2.000000,3
5,2023-10-19,AMR Tier 1 iOS Phone Spanish TP (Colombia...,AMR Tier 1 iOS Phone Spanish,TP (Colombia AHA),0.819,24.89,0.9211,35701,4,3,5,4.000000,5
6,2023-10-19,AMR Tier 1 Mac+ Phone Spanish Apple (Austin),AMR Tier 1 Mac+ Phone Spanish,Apple (Austin),0.8276,23.07,0.9167,2824,3,5,2,3.333333,3
7,2023-10-19,AMR Tier 1 Mac+ Phone Spanish Apple (Elk ...,AMR Tier 1 Mac+ Phone Spanish,Apple (Elk Grove),0.7656,22.72,0.9362,4063,6,3,1,3.333333,3
8,2023-10-19,AMR Tier 1 Mac+ Phone Spanish Apple (US AHA),AMR Tier 1 Mac+ Phone Spanish,Apple (US AHA),0.85,24.43,NaN,1023,1,-1,4,1.333333,1
9,2023-10-19,AMR Tier 1 Mac+ Phone Spanish Concentrix ...,AMR Tier 1 Mac+ Phone Spanish,Concentrix (Managua),0.8064,23.31,0.9617,14317,4,1,3,2.666667,2


In [ ]:
bogota_data = combined_df[combined_df['Staff Type | ST'] == 'TP (Colombia AHA)']
bogota_data

,Date_File,LOB+site,LOB,Staff Type | ST,CSAT,CAHT,QA,Volumen,CSAT Ranking,QA Ranking,CAHT Ranking,Total Score,SRS R
5,2023-10-19,AMR Tier 1 iOS Phone Spanish TP (Colombia...,AMR Tier 1 iOS Phone Spanish,TP (Colombia AHA),0.819,24.89,0.9211,35701,4,3,5,4.000000,5
12,2023-10-19,AMR Tier 1 Mac+ Phone Spanish TP (Colombi...,AMR Tier 1 Mac+ Phone Spanish,TP (Colombia AHA),0.8497,25.45,0.875,5616,2,6,7,5.000000,5
22,2023-10-19,WW Tier 1 iOS Chat English TP (Colombia AHA),WW Tier 1 iOS Chat English,TP (Colombia AHA),0.7011,28.74,0.8959,63702,10,9,10,9.666667,8
30,2023-10-19,AMR Tier 2 Mac+ Phone Spanish TP (Colombi...,AMR Tier 2 Mac+ Phone Spanish,TP (Colombia AHA),0.7569,20.96,0.9512,3611,4,2,4,3.333333,2
40,2023-10-19,WW Tier 1 Mac+ Chat English TP (Colombia ...,WW Tier 1 Mac+ Chat English,TP (Colombia AHA),0.7824,26.75,0.9071,43540,7,10,3,6.666667,6
44,2023-10-19,WW Tier 1 Mac+ Chat Spanish TP (Colombia ...,WW Tier 1 Mac+ Chat Spanish,TP (Colombia AHA),0.7994,26.28,0.9202,47268,2,2,1,1.666667,2


In [ ]:
def aplicar_formula(group):
    group["SRS CSAT"] = (group["CSAT Ranking"] / group["CSAT Ranking"].count()) - (1 / group["CSAT Ranking"].count()) / 2
    group["SRS QA"] = (group["QA Ranking"] / group["QA Ranking"].count()) - (1 / group["QA Ranking"].count()) / 2
    group["SRS CAHT"] = (group["CAHT Ranking"] / group["CAHT Ranking"].count()) - (1 / group["CAHT Ranking"].count()) / 2
    group["SRS Real"] = (group["SRS R"] / group["SRS R"].count()) - (1 / group["SRS R"].count()) / 2
    return group

# Aplicar la fórmula a cada grupo basado en los valores únicos de "LOB"
df_srs = combined_df.groupby("LOB").apply(aplicar_formula).reset_index(drop=True)

<ipython-input-8-4592892e7f03>:9: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_srs = combined_df.groupby("LOB").apply(aplicar_formula).reset_index(drop=True)


In [ ]:
bogota_data = df_srs[df_srs['LOB'] == 'AMR Tier 2 Mac+ Phone Spanish']
bogota_data

,Date_File,LOB+site,LOB,Staff Type | ST,CSAT,CAHT,QA,Volumen,CSAT Ranking,QA Ranking,CAHT Ranking,Total Score,SRS R,SRS CSAT,SRS QA,SRS CAHT,SRS Real
24,2023-10-19,AMR Tier 2 Mac+ Phone Spanish Apple (Austin),AMR Tier 2 Mac+ Phone Spanish,Apple (Austin),0.5862,18.33,0.9155,729,7,6,1,4.666667,5,0.928571,0.785714,0.071429,0.642857
25,2023-10-19,AMR Tier 2 Mac+ Phone Spanish Apple (Elk ...,AMR Tier 2 Mac+ Phone Spanish,Apple (Elk Grove),0.7077,21.04,0.8448,1848,6,7,5,6.000000,6,0.785714,0.928571,0.642857,0.785714
26,2023-10-19,AMR Tier 2 Mac+ Phone Spanish Apple (US AHA),AMR Tier 2 Mac+ Phone Spanish,Apple (US AHA),0.7102,19.7,0.9165,5992,5,5,2,4.000000,4,0.642857,0.642857,0.214286,0.500000
27,2023-10-19,AMR Tier 2 Mac+ Phone Spanish Concentrix ...,AMR Tier 2 Mac+ Phone Spanish,Concentrix (Managua),0.8041,20.19,0.9194,8145,1,4,3,2.666667,1,0.071429,0.500000,0.357143,0.071429
28,2023-10-19,AMR Tier 2 Mac+ Phone Spanish Concentrix ...,AMR Tier 2 Mac+ Phone Spanish,Concentrix (San Jose HS),0.7706,21.52,0.9421,6006,2,3,6,3.666667,3,0.214286,0.357143,0.785714,0.357143
29,2023-10-19,AMR Tier 2 Mac+ Phone Spanish TP (Bogota),AMR Tier 2 Mac+ Phone Spanish,TP (Bogota),0.7692,22.14,0.9528,3567,3,1,7,3.666667,3,0.357143,0.071429,0.928571,0.357143
30,2023-10-19,AMR Tier 2 Mac+ Phone Spanish TP (Colombi...,AMR Tier 2 Mac+ Phone Spanish,TP (Colombia AHA),0.7569,20.96,0.9512,3611,4,2,4,3.333333,2,0.500000,0.214286,0.500000,0.214286
